In [3]:
import os
import pandas as pd
import numpy as np
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import _gradient_boosting
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from helper_functions_pipe_testing import *
from sklearn.metrics import  f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import *
from sklearn.preprocessing import LabelEncoder
from helper_functions_KP import *
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
import math

#Instructions for the pipeline Requires two inputs for training: - Mass spec data with corresponding NP surface characteristics and experimental conditions (time, concentration) - NetsurfP and Biopython data that has been precalculated - X characteristics to predict
pipeline Take mass spec spreadsheet Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration Merge with Proteome data to get file that has Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration,Mass,Length,Sequence Calculate protein features using biopython Merge with NSP data to get all protein features
Split into X and Y dataset with Entries as labels

In [33]:
### Make 3 DFs### Load saved DFs instead of running this cell
#DF 1 All_ files combined
#DF 2 Synth Files ( the 19 UniqueNPs that I ran with my synthesized NPs+some PS NPs)
#DF 3 All_files 30% zeros
multi_files = True  #set to false if you just want to set one  prot_abund_file
in_dir = "Input_data/Proteomic data/Abundance2/"
prot_abund_file = 'Input_data/Proteomic data/abundance/Norm_Intensity_Bovsynth022123.xlsx'
NP_filepath = 'Input_data/NPs/NP_Database2.xlsx'
controls_file = 'Input_data/Proteomic data/controls_combined.xlsx'
uniprot_filepath = 'Input_data/BioPython_data/Combined_biopyCalcs.xlsx'
NSPfilePath = 'Input_data/NetSurfP_data/Combined.xlsx'
cutoff_vals=[0.1,0.2,0.3,0.5]
files = os.listdir(in_dir)
for zerosperrow in cutoff_vals:
    if multi_files == True:
        for i, f in enumerate(files):
            if i == 0:
                raw_MS_data = pd.read_excel(in_dir + f, header=0)
                cols = raw_MS_data.shape[1]
                cutoff = int(zerosperrow * cols)
                print('shape before dropping rows', raw_MS_data.shape)
                raw_MS_data = raw_MS_data.drop(raw_MS_data[(raw_MS_data == 0).sum(axis=1) >= cutoff].index)
                print('shape after dropping rows', raw_MS_data.shape)
                # print(raw_MS_data)
                raw_MS_data = pd.melt(raw_MS_data, id_vars=['Entry'], var_name='Sample_num', value_name='Abundance')

            else:

                temp = pd.read_excel(in_dir + f, header=0)
                cols = temp.shape[1]
                cutoff = int(zerosperrow * cols)
                print('shape before dropping rows', raw_MS_data.shape)
                temp = temp.drop(temp[(temp == 0).sum(axis=1) >= cutoff].index)
                print('shape after dropping rows', raw_MS_data.shape)
                temp = pd.melt(temp, id_vars=['Entry'], var_name='Sample_num', value_name='Abundance')

                raw_MS_data = pd.concat([raw_MS_data, temp])
                print('final shape after melt', raw_MS_data.shape)
                print('number of zeros in the dataset:',(raw_MS_data == 0).sum().sum())


    else:
        raw_MS_data = pd.read_excel(prot_abund_file, header=0)
        cols = raw_MS_data.shape[1]
        cutoff = int(zerosperrow * cols)
        print('shape beofre dropping rows', raw_MS_data.shape)
        raw_MS_data = raw_MS_data.drop(raw_MS_data[(raw_MS_data == 0).sum(axis=1) >= cutoff].index)
        print('shape after dropping rows', raw_MS_data.shape)
        raw_MS_data = pd.melt(raw_MS_data, id_vars=['Entry'], var_name='Sample_num', value_name='Abundance')
    #remove prots that were added due to merge
    raw_MS_data = raw_MS_data.dropna()
    ###Bring in controls (MS data for serums)##
    controls = pd.read_excel(controls_file, header=0)
    MS_data_controls = pd.merge(raw_MS_data, controls, how='left', on='Entry')
    ###Bring in Uniprot_data,NSPdata and NP data##
    uniprot_dat = pd.read_excel(uniprot_filepath, header=0)
    NSP_data = pd.read_excel(NSPfilePath)
    ###Bring in NP data and merge to get complete NP dataset###
    NPUNdata = pd.read_excel(NP_filepath, header=0, sheet_name='NPUNID')
    NPprop = pd.read_excel(NP_filepath, header=0, sheet_name='NP_Props')
    NPdata = pd.merge(NPUNdata, NPprop, how="left", on='NPID')
    # NPdata.dropna(inplace=True)
    #calculate Enrichment
    #####MAYBE add binning here to keep negative results and improve capapbilities######
    # MS_data_controls['Enrichment']= np.log2(MS_data_controls['Abundance']/MS_data_controls['Abundance_Controls'])
    # MS_data=MS_data_controls.drop(columns=['Abundance','Abundance_Controls'])
    raw_prop_data = pd.merge(MS_data_controls, uniprot_dat.drop_duplicates(subset=['Entry']), how='left', on='Entry')
    Protein_data_complete = pd.merge(raw_prop_data, NSP_data.drop_duplicates(subset=['Entry']), how='left',
                                 on='Entry')
    #merges netsurfp features and biopython features
    Protein_data_complete.fillna(0, inplace=True)
    # print('Sample Nums in Protein Data',Protein_data_complete['Sample_num'])
    data_complete = pd.merge(Protein_data_complete, NPdata, how='inner', on='Sample_num')
    data_complete.fillna(0, inplace=True)
    #create ordinal variables for the core materials and surface ligands
    le = LabelEncoder()
    data_complete['Core Material'] = le.fit_transform(data_complete['Core Material'])
    data_complete['Surface_Ligand'] = le.fit_transform(data_complete['Surface_Ligand'])
    #shuffle data using sample
    data_complete = data_complete.sample(frac=1)

    data_complete.drop(columns=['notes', 'Notes'], inplace=True)

    data_complete.to_excel("Input_data/Save_files/df_3_all_"+str(int(zerosperrow*100))+"%zeros.xlsx", index=False)


shape before dropping rows (474, 14)
shape after dropping rows (7, 14)
shape before dropping rows (91, 3)
shape after dropping rows (91, 3)
final shape after melt (3061, 3)
number of zeros in the dataset: 65
shape before dropping rows (474, 14)
shape after dropping rows (13, 14)
shape before dropping rows (169, 3)
shape after dropping rows (169, 3)
final shape after melt (4844, 3)
number of zeros in the dataset: 298
shape before dropping rows (474, 14)
shape after dropping rows (27, 14)
shape before dropping rows (351, 3)
shape after dropping rows (351, 3)
final shape after melt (6841, 3)
number of zeros in the dataset: 752
shape before dropping rows (474, 14)
shape after dropping rows (52, 14)
shape before dropping rows (676, 3)
shape after dropping rows (676, 3)
final shape after melt (10191, 3)
number of zeros in the dataset: 2066


In [ ]:
# Large OG Pipeline # Move lower #

#Editable Variables
#list of test filters
# zerosperrow = 0.3
# multi_files = True  #set to false if you just want to set one  prot_abund_file
# rfecv_ = True  #True_runs RFECV
# rfe_ = False     #True runs Recursive feature elimination
# abund_controls = True # True keeps the serum as an input feature
# splits = 10     #number of splits for cross validation across QC methods and feature selection methods
# in_dir = "Input_data/Proteomic data/Abundance2/"
# prot_abund_file = 'Input_data/Proteomic data/Abundance2/Norm_Intensity _all20230403.xlsx'
# NP_filepath = 'Input_data/NPs/NP_Database.xlsx'
# controls_file = 'Input_data/Proteomic data/controls_combined.xlsx'
# uniprot_filepath = 'Input_data/BioPython_data/Combined_biopyCalcs.xlsx'
# NSPfilePath = 'Input_data/NetSurfP_data/Combined.xlsx'
#
# scorings=['r2','neg_mean_squared_error','mean_absolute_error']
# RFE_Feats = 40
# model = RandomForestRegressor(n_estimators=80)
# # model=XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, objective='reg:squarederror')
# summary_tmp=[]
# for z in scorings:
#     id = 'Con_drop_' +str(z)+ '_30' + '%zeros'
#     # take files in_dir and combine then into a pandas df (raw_MS_data) ###USE when combining multiple datasets####
#     # melt the df to make it an accession number, NPUNID, Abundance dataset before combining
#     files = os.listdir(in_dir)
#     if multi_files == True:
#         for i, f in enumerate(files):
#             if i == 0:
#                 raw_MS_data = pd.read_excel(in_dir + f, header=0)
#                 cols = raw_MS_data.shape[1]
#                 cutoff = int(zerosperrow * cols)
#                 print('shape before dropping rows', raw_MS_data.shape)
#                 raw_MS_data = raw_MS_data.drop(raw_MS_data[(raw_MS_data == 0).sum(axis=1) >= cutoff].index)
#                 print('shape after dropping rows', raw_MS_data.shape)
#                 # print(raw_MS_data)
#                 raw_MS_data = pd.melt(raw_MS_data, id_vars=['Entry'], var_name='Sample_num', value_name='Abundance')
#
#             else:
#
#                 temp = pd.read_excel(in_dir + f, header=0)
#                 cols = temp.shape[1]
#                 cutoff = int(zerosperrow * cols)
#                 print('shape before dropping rows', raw_MS_data.shape)
#                 temp = temp.drop(temp[(temp == 0).sum(axis=1) >= cutoff].index)
#                 print('shape after dropping rows', raw_MS_data.shape)
#                 temp = pd.melt(temp, id_vars=['Entry'], var_name='Sample_num', value_name='Abundance')
#
#                 raw_MS_data = pd.concat([raw_MS_data, temp])
#                 print('final shape after melt', raw_MS_data.shape)
#                 print('number of zeros in the dataset:',(raw_MS_data == 0).sum().sum())
#
#
#     else:
#         raw_MS_data = pd.read_excel(prot_abund_file, header=0)
#         cols = raw_MS_data.shape[1]
#         cutoff = int(zerosperrow * cols)
#         print('shape beofre dropping rows', raw_MS_data.shape)
#         raw_MS_data = raw_MS_data.drop(raw_MS_data[(raw_MS_data == 0).sum(axis=1) >= cutoff].index)
#         print('shape after dropping rows', raw_MS_data.shape)
#         raw_MS_data = pd.melt(raw_MS_data, id_vars=['Entry'], var_name='Sample_num', value_name='Abundance')
#     #remove prots that were added due to merge
#     raw_MS_data = raw_MS_data.dropna()
#     ###Bring in controls (MS data for serums)##
#     controls = pd.read_excel(controls_file, header=0)
#     MS_data_controls = pd.merge(raw_MS_data, controls, how='left', on='Entry')
#     ###Bring in Uniprot_data,NSPdata and NP data##
#     uniprot_dat = pd.read_excel(uniprot_filepath, header=0)
#     NSP_data = pd.read_excel(NSPfilePath)
#     ###Bring in NP data and merge to get complete NP dataset###
#     NPUNdata = pd.read_excel(NP_filepath, header=0, sheet_name='NPUNID')
#     NPprop = pd.read_excel(NP_filepath, header=0, sheet_name='NP_Props')
#     NPdata = pd.merge(NPUNdata, NPprop, how="left", on='NPID')
#     NPdata.dropna(inplace=True)
#     #calculate Enrichment
#     #####MAYBE add binning here to keep negative results and improve capapbilities######
#     # MS_data_controls['Enrichment']= np.log2(MS_data_controls['Abundance']/MS_data_controls['Abundance_Controls'])
#     # MS_data=MS_data_controls.drop(columns=['Abundance','Abundance_Controls'])
#     raw_prop_data = pd.merge(MS_data_controls, uniprot_dat.drop_duplicates(subset=['Entry']), how='left', on='Entry')
#     Protein_data_complete = pd.merge(raw_prop_data, NSP_data.drop_duplicates(subset=['Entry']), how='left',
#                                      on='Entry')
#     #merges netsurfp features and biopython features
#     Protein_data_complete.fillna(0, inplace=True)
#     data_complete = pd.merge(Protein_data_complete, NPdata, how='inner', on='Sample_num')
#     data_complete.fillna(0, inplace=True)
#     #create ordinal variables for the core materials and surface ligands
#     le = LabelEncoder()
#     data_complete['Core Material'] = le.fit_transform(data_complete['Core Material'])
#     data_complete['Surface_Ligand'] = le.fit_transform(data_complete['Surface_Ligand'])
#     #shuffle data using sample
#     data_complete = data_complete.sample(frac=1)
#     #set labels (what we are trying to predict) as Abundance
#     label_abund_df = data_complete['Abundance'].copy()
#     label_abund = np.ravel(label_abund_df)
#     NPIDs=data_complete['NPUNID'].copy()
#     data_complete.drop(columns=['notes', 'Notes', 'NPUNID'], inplace=True)
#
#     #make it one dimenisional
#     #drop qualitative, not neccessary, and label columns
#     #create df without bonus NSP columns (remove total_exposed) There are too sets of features total_exposed and exposed_exposed
#     to_drop = data_complete.filter(like='total_exposed_')
#     data_complete.drop(columns=to_drop, inplace=True)
#     df = data_complete.drop(
#         ['Entry', 'Abundance', 'Sequence', 'NPID', 'Ligands', 'Protein Source', 'Sample_num', 'Unnamed: 5',
#          'Raw_FileID'], axis=1)
#     if abund_controls == False:
#         df.drop(columns=['Abundance_Controls'], inplace=True)
#
#     df_out=pd.concat([df, NPIDs,label_abund_df],axis=1)
#     df_out.to_excel("Input_data/Save_files/df_whole_"+id+".xlsx")

In [6]:
#Sample code for loading saved DF and dropping appropriate columns before running### Also For running RFECV and saving the reduced df
df_filepath='Input_data/Save_files/df_1_all.xlsx'
id='04232023'
df = pd.read_excel(df_filepath, header=0)
labels_df= df['Abundance'].copy()
id_col= df['NPUNID'].copy()
label_abund=np.ravel(labels_df)
model=RandomForestRegressor(n_estimators=100)
splits=10
scoring='neg_mean_squared_error'
df=df.drop(columns=['Entry', 'Sequence', 'NPID', 'Ligands', 'Protein Source', 'Sample_num', 'Unnamed: 5','Raw_FileID'])
import math

start = 0.0001
stop = 1
num_points = 10
# Calculate the common difference between consecutive terms in the series
common_diff = (math.log(stop) - math.log(start)) / (num_points - 1)
# Generate the logarithmic series as a list
Abund_filter = [start * math.exp(i * common_diff) for i in range(num_points)]
# model = RandomForestRegressor(n_estimators=80)
# summary_tmp=[]
for i in Abund_filter:
    df_out= df[df['Abundance']>=i].copy()
# df = RFECV_plot(df,label_abund,model,id,folds=splits,step=2,scoring=scoring)

    df_out.to_excel("Input_data/Save_files/Abund_Filters/df_1_all"+str(i)+".xlsx")


In [ ]:
## Run RFECV through each different condition (zeros and abund filters)##

in_dir='Input_data/Save_files/ZerosReduced/'
files=os.listdir(in_dir)
files
for file in files:
    filepath=in_dir+file
    id=file[:-10]
    # print(id)
    # print(filepath)
    df= pd.read_excel(filepath,header=0)
    labels_df= df['Abundance'].copy()
    id_col= df['NPUNID'].copy()
    label_abund=np.ravel(labels_df)
    model=RandomForestRegressor(n_estimators=100)
    splits=5
    df=df.drop(columns=['Entry', 'Abundance', 'Sequence', 'NPID', 'Ligands', 'Protein Source', 'Sample_num', 'Unnamed: 5','Raw_FileID','NPUNID','Abundance_Controls'])
    df = RFECV_plot(df,label_abund,model,id,folds=splits,step=2)
    df_out=pd.concat([df, id_col,labels_df], axis=1)
    df_out.to_excel(in_dir+id+'RFECV.xlsx',index=False)



In [15]:
model=RandomForestRegressor(n_estimators=100)
df_filepath='Input_data/Save_files/df_1_all.xlsx'
df = pd.read_excel(df_filepath, header=0)
summary_tmp=[]
feats=[]
for i in range(1,4):
    df_test=df[df['NPUNID']==i].copy()
    labels_df= df_test['Abundance'].copy()
    id_col= df_test['NPUNID'].copy()
    label_abund=np.ravel(labels_df)
    df_test=df_test.drop(columns=['Entry', 'Abundance', 'Sequence', 'NPID', 'Ligands', 'Protein Source', 'Sample_num', 'Unnamed: 5','Raw_FileID','NPUNID','Abundance_Controls'])
    step = 3
    estimator = model
    selector = RFE(estimator, n_features_to_select=15, step=step)
    selector = selector.fit(df_test, label_abund)
    feat_list = selector.get_feature_names_out()
    df_test = df_test[feat_list].copy()


    model.fit(df_test,labels_df)
    tmp2,tmp3=scorer(df_test, label_abund, model, str(i), 10)
    summary_tmp.append(tmp2)
    feats.append(tmp3)
# lasso_feature_selection(df, label_abund, id)
summary=pd.concat(summary_tmp,axis=0)
feat_summary=feats[0]
for d in feats[1:]:
    feat_summary=feat_summary.merge(d, on='Features',how='outer')



Scorer ran successfully
Scorer ran successfully
Scorer ran successfully


,Features,Importance1,Features,Importance2,Features,Importance3
0,Mass,0.048258,Mass,0.077269,Mass,0.056966
1,frac_aa_C,0.089188,frac_aa_C,0.083629,frac_aa_C,0.134803
2,frac_aa_R,0.052345,frac_aa_W,0.061838,frac_aa_R,0.022273
3,frac_aa_Y,0.068751,frac_aa_Y,0.029503,frac_aa_W,0.031412
4,molecular_weight,0.042757,molecular_weight,0.066677,frac_aa_Y,0.019674
5,flexibility_min,0.063276,flexibility_min,0.116857,molecular_weight,0.064382
6,asa_sum,0.073286,secondary_structure_fraction_disordered,0.054458,flexibility_min,0.104326
7,fraction_total_exposed_G,0.093989,fraction_exposed,0.053183,secondary_structure_fraction_sheet,0.043438
8,fraction_total_exposed_M,0.046700,asa_sum,0.058161,asa_sum,0.043868
9,fraction_exposed_exposed_H,0.049535,fraction_total_exposed_G,0.131658,fraction_total_exposed_G,0.277558


In [20]:
feat_summary=feats[0]
for d in feats[1:]:
    feat_summary=feat_summary.merge(d, on='Features',how='outer')
feat_summary

,Features,Importance1,Importance2,Importance3
0,Mass,0.048258,0.077269,0.056966
1,frac_aa_C,0.089188,0.083629,0.134803
2,frac_aa_R,0.052345,NaN,0.022273
3,frac_aa_Y,0.068751,0.029503,0.019674
4,molecular_weight,0.042757,0.066677,0.064382
5,flexibility_min,0.063276,0.116857,0.104326
6,asa_sum,0.073286,0.058161,0.043868
7,fraction_total_exposed_G,0.093989,0.131658,0.277558
8,fraction_total_exposed_M,0.046700,NaN,NaN
9,fraction_exposed_exposed_H,0.049535,0.042334,0.059812


In [14]:
df['NPUNID'].unique()

array([12, 40, 44, 33,  7,  2, 43, 17, 27, 20, 49,  3, 48,  9, 45, 32, 29,
       28, 23, 47, 18, 25, 26, 14, 19,  1,  8, 38, 15, 24, 41, 21, 11, 22,
       39, 34, 37, 31, 42, 35, 46, 50, 10, 36, 16, 30, 13], dtype=int64)

In [9]:
feat_import=model.feature_importances_
feat_scores=list(zip(df_test.columns.tolist(),feat_import))
# feat_scores
feats=pd.DataFrame(feat_scores,columns=['Feature','Feature Importance'])
feats


,Feature,Feature Importance
0,Length,0.026158
1,Mass,0.031579
2,frac_aa_A,0.003928
3,frac_aa_C,0.043846
4,frac_aa_D,0.002819
...,...,...
108,ProteinID,0.000000
109,NP_incubation Concentration (mg/mL),0.000000
110,Incubation Concentration (mg/ml),0.000000
111,Incubation Time (minutes),0.000000


In [3]:
df_filepath='Input_data/Save_files/df_1_all.xlsx'
id='04232023'
df = pd.read_excel(df_filepath, header=0)
labels_df= df['Abundance'].copy()
id_col= df['NPUNID'].copy()
label_abund=np.ravel(labels_df)
model=RandomForestRegressor(n_estimators=100)
print('here')
df=df.drop(columns=['Entry', 'Abundance', 'Sequence', 'NPID', 'Ligands', 'Protein Source', 'Sample_num', 'Unnamed: 5','Raw_FileID','NPUNID','Abundance_Controls'])

selector = RFE(model, n_features_to_select=50, step=3)
print('start')
selector = selector.fit(df, label_abund)
print('fit complete')
selector.support_
ranking = selector.ranking_
feat_list = selector.get_feature_names_out()
df = df[feat_list]
df_out=pd.concat([df, id_col,labels_df],axis=1)
df_out.to_excel("Input_data/Save_files/df_1_all_RFE50_RFR.xlsx",index=False)

NameError: name 'pd' is not defined

In [ ]:
# Standard Pipeline###


# Data to record#
#MSE,Pearson,R2,number of features => Scorer
#Feature importance/Features used=> Scorer, but send out separate df for those

#Editable Variables
#list of test filters
rfecv_ = True  #True_runs RFECV
rfe_ = False     #True runs Recursive feature elimination
abund_controls = False # True keeps the serum as an input feature
splits = 10     #number of splits for cross validation across QC methods and feature selection methods
# scorings=['r2','neg_mean_squared_error','mean_absolute_error']
# RFE_Feats = 40
model = RandomForestRegressor(n_estimators=80)

df_filepath='Input_data/Save_files/df_1_all_RFE50_RFR.xlsx'
df = pd.read_excel(df_filepath, header=0)
labels_df= df['Abundance'].copy()
id_col= df['NPUNID'].copy()
label_abund=np.ravel(labels_df)
NPIDs=np.ravel(id_col)
id='RFR_RFECV_100%'
# df=df.drop(columns=['Entry', 'Abundance', 'Sequence', 'NPID', 'Ligands', 'Protein Source', 'Sample_num', 'Unnamed: 5','Raw_FileID','NPUNID'])
df=df.drop(columns=['Abundance','NPUNID'],axis=1)
# if abund_controls == False:
#     df.drop(columns=['Abundance_Controls'], inplace=True)

# model=XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, objective='reg:squarederror')
summary_tmp=[]
# for z in scorings:
    #Run PCA to seee how data differentiates#
PCA_plot(df,NPIDs,id)
#use recursive feature elimination with Random Forest Regression as the estimator to select top 45 important features
if rfe_ == True:
    step = 2
    estimator = RandomForestRegressor(n_estimators=100)
    selector = RFE(estimator, n_features_to_select=RFE_Feats, step=step)
    selector = selector.fit(df, label_abund)
    selector.support_
    ranking = selector.ranking_
    feat_list = selector.get_feature_names_out()
    df = df[feat_list]
    df_out=pd.concat([df, NPIDs,labels_df],axis=1)
    df_out.to_excel("Input_data/Save_files/df_RFE40_"+id+".xlsx")

#run Recursive feature elimination with cross validation
if rfecv_ == True:
    df = RFECV_plot(df,label_abund,model,id,folds=splits,step=2)
    df_out=pd.concat([df, NPIDs,labels_df],axis=1)
    df_out.to_excel("Input_data/Save_files/df_RFECV(50)_"+id+".xlsx")
#Quality control
scram_score(df, label_abund, model, id, 0.2)
# feat_drop(df, label_abund, model, id, 0.2)
feat_drop_multifold(df, label_abund, model, id, 0.2, folds=splits)
tmp2=scorer(df, label_abund, model, id, 10)
zeros=(raw_MS_data['Abundance']==0).sum()
percent_zeros=zeros/raw_MS_data.shape[0]
tmp2['TotalZeros']=zeros
tmp2['Percent_zeros']=percent_zeros
summary_tmp.append(tmp2)
# lasso_feature_selection(df, label_abund, id)
summary=pd.concat(summary_tmp,axis=0)
summary.to_excel('Output_data/'+id+'.xlsx', index=False)
print('done')


PCA Ran successfully


In [11]:
### Running Binary Classification systems and seeing what abund threshold is the best for performance##

abund_thresh=np.arange(0.01,1.0,0.05).tolist()
df_filepath='Input_data/Save_files/df_1_all.xlsx'
df = pd.read_excel(df_filepath, header=0)
model=RandomForestClassifier()
# id='RFR_AFilter_'+str(i)+'NoCon'
# Create an empty pandas DataFrame to store the evaluation metrics
eval_df = pd.DataFrame(columns=['Abund Thresh', 'F1', 'AUROC', 'Accuracy', 'Precision', 'Recall'])
df=df.drop(columns=['Entry', 'Sequence', 'NPID', 'Ligands', 'Protein Source', 'Sample_num', 'Unnamed: 5','Raw_FileID'])
df=df.drop(columns=['Abundance_Controls'])

# Iterate through multiple iterations of model training and testing
for i in abund_thresh:
    df_a=df.copy()
    df_a['binary_target']= df_a['Abundance'].apply(lambda t: 1 if t>=i else 0)
    labels_df = df_a['binary_target'].copy()
    # id_col= df_a['NPUNID'].copy()
    label_binary=np.ravel(labels_df)
    df_a=df_a.drop(columns=['Abundance','NPUNID','binary_target']).copy()

    step = 2
    RFE_Feats=15
    selector = RFE(model, n_features_to_select=RFE_Feats, step=step)
    selector = selector.fit(df_a, label_binary)
    selector.support_
    ranking = selector.ranking_
    feat_list = selector.get_feature_names_out()
    df_a = df_a[feat_list].copy()

    x_train, x_test, y_train, y_test = train_test_split(df_a, label_binary, test_size=0.2, random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    f1 = f1_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Append a new row to the eval_df DataFrame with the evaluation metrics for this iteration
    eval_df = eval_df.append({'Abund Thresh': i,
                              'F1': f1,
                              'AUROC': auroc,
                              'Accuracy': accuracy,
                              'Precision': precision,
                              'Recall': recall}, ignore_index=True)

eval_df.to_excel('Output_data/AbundThresholdingRFC.xlsx', index=False)



C:\Users\pouls\AppData\Local\Temp\ipykernel_13848\1895467353.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eval_df = eval_df.append({'Abund Thresh': i,
C:\Users\pouls\AppData\Local\Temp\ipykernel_13848\1895467353.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eval_df = eval_df.append({'Abund Thresh': i,
C:\Users\pouls\AppData\Local\Temp\ipykernel_13848\1895467353.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eval_df = eval_df.append({'Abund Thresh': i,
C:\Users\pouls\AppData\Local\Temp\ipykernel_13848\1895467353.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eval_df = eval_df.append({'Abund Thresh': i,
C:\Users

In [ ]:
###Filtering Abundance to see if i stick to higher abundance I get better performance##
rfecv_ = False  #True_runs RFECV
rfe_ = True     #True runs Recursive feature elimination
abund_controls = False # True keeps the serum as an input feature
splits = 10     #number of splits for cross validation across QC methods and feature selection methods

df_filepath='Input_data/Save_files/df_1_all.xlsx'
id='04232023'
df = pd.read_excel(df_filepath, header=0)
labels_df= df['Abundance'].copy()
id_col= df['NPUNID'].copy()
label_abund=np.ravel(labels_df)
model=RandomForestRegressor(n_estimators=100)
print('here')
df=df.drop(columns=['Entry', 'Abundance', 'Sequence', 'NPID', 'Ligands', 'Protein Source', 'Sample_num', 'Unnamed: 5','Raw_FileID','NPUNID','Abundance_Controls'])

df_filepath = 'Input_data/Save_files/df_1_all.xlsx'
df_a = pd.read_excel(df_filepath, header=0)
df_a=df_a.drop(columns=['Entry', 'Sequence', 'NPID', 'Ligands', 'Protein Source', 'Sample_num', 'Unnamed: 5','Raw_FileID'])
# scorings=['r2','neg_mean_squared_error','neg_mean_absolute_error']
RFE_Feats = 15
import math

start = 0.0001
stop = 1
num_points = 15

# Calculate the common difference between consecutive terms in the series
common_diff = (math.log(stop) - math.log(start)) / (num_points - 1)

# Generate the logarithmic series as a list
Abund_filter = [start * math.exp(i * common_diff) for i in range(num_points)]
model = RandomForestRegressor(n_estimators=80)
summary_tmp=[]
for i in Abund_filter:
    df= df_a[df_a['Abundance']>=i]
    id='RFR_AFilter_'+str(i)+'NoCon'
    labels_df= df['Abundance'].copy()
    id_col= df['NPUNID'].copy()
    label_abund=np.ravel(labels_df)
    df.drop(columns=['Abundance','NPUNID'],inplace=True)
    if abund_controls == False:
        df.drop(columns=['Abundance_Controls'], inplace=True)
    #run Recursive feature elimination with cross validation
    if rfecv_ == True:
        df = RFECV_plot(df,label_abund,model,id,folds=splits,step=2,scoring=z)
        # df_out=pd.concat([df, NPIDs,label_abund_df],axis=1)
        # df_out.to_excel("Input_data/Save_files/df_RFECV(40)_"+id+".xlsx")
    #Run Recursive feature elimination to remove down to RFE_Feats
    if rfe_ == True:
        step = 2
        selector = RFE(model, n_features_to_select=RFE_Feats, step=step)
        selector = selector.fit(df, label_abund)
        selector.support_
        ranking = selector.ranking_
        feat_list = selector.get_feature_names_out()
        df = df[feat_list]
        # df_out=pd.concat([df, NPIDs,label_abund_df],axis=1)
        # df_out.to_excel("Input_data/Save_files/df_RFE40_"+id+".xlsx")
    tmp2=scorer(df, label_abund, model, id, 10)
    tmp2['Abundance Filter']=i

    summary_tmp.append(tmp2)

    summary=pd.concat(summary_tmp,axis=0)
summary.to_excel('Output_data/'+id+'.xlsx', index=False)


In [ ]:
df_a = pd.read_excel(df_filepath, header=0)
# df=df_a[df_a['Abundance']>=0.01]
# df_a['Abundance']=np.log10(df_a['Abundance']+1)
df['Abundance']=np.log2(df['Abundance']+1)
plt.hist(df['Abundance'],bins=25)
plt.xlim(-2,2)
plt.show()

In [ ]:
#Editable Variables
#list of test filters
zerosperrow = 0.3
multi_files = True  #set to false if you just want to set one  prot_abund_file
rfecv_ = True  #True_runs RFECV
rfe_ = False     #True runs Recursive feature elimination
abund_controls = True # True keeps the serum as an input feature
splits = 10     #number of splits for cross validation across QC methods and feature selection methods
in_dir = "Input_data/Proteomic data/Abundance2/"
prot_abund_file = 'Input_data/Proteomic data/Abundance2/Norm_Intensity _all20230403.xlsx'
NP_filepath = 'Input_data/NPs/NP_Database.xlsx'
controls_file = 'Input_data/Proteomic data/controls_combined.xlsx'
uniprot_filepath = 'Input_data/BioPython_data/Combined_biopyCalcs.xlsx'
NSPfilePath = 'Input_data/NetSurfP_data/Combined.xlsx'
df_filepath='Input_data/Save_files/df_whole_Con_drop_30%zeros.xlsx'
df = pd.read_excel(df_filepath, header=0)
scorings=['r2','neg_mean_squared_error','neg_mean_absolute_error']
RFE_Feats = 40
model = RandomForestRegressor(n_estimators=80)
# model=XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, objective='reg:squarederror')
summary_tmp=[]
for z in scorings:
    df=pd.read_excel('Input_data/Save_files/df_whole_noCon_drop30%zeros.xlsx')
    labels_df= df['label_abund_df'].copy()
    id_col= df['NPIDs'].copy()
    labels=np.ravel(labels_df)
    df.drop(columns=['Abundance','NPUNID'],inplace=True)
    # label_abund_df.to_excel("Input_data/Save_files/label_abund"+id+".xlsx",index=False)
    # Run PCA to seee how data differentiates#
    # PCA_plot(df,label_abund,id)


    #run Recursive feature elimination with cross validation
    if rfecv_ == True:
        df = RFECV_plot(df,label_abund,model,id,folds=splits,step=2,scoring=z)
        df_out=pd.concat([df, NPIDs,label_abund_df],axis=1)
        df_out.to_excel("Input_data/Save_files/df_RFECV(40)_"+id+".xlsx")

    #use recursive feature elimination with Random Forest Regression as the estimator to select top 45 important features
    if rfe_ == True:
        step = 2
        estimator = RandomForestRegressor(n_estimators=100)
        selector = RFE(estimator, n_features_to_select=RFE_Feats, step=step)
        selector = selector.fit(df, label_abund)
        selector.support_
        ranking = selector.ranking_
        feat_list = selector.get_feature_names_out()
        df = df[feat_list]
        df_out=pd.concat([df, NPIDs,label_abund_df],axis=1)
        df_out.to_excel("Input_data/Save_files/df_RFE40_"+id+".xlsx")

    #Quality control
    v
# summary.to_excel('Output_data/'+id+'.xlsx', index=False)
print('done')

In [25]:
##Testing Each NP independently## Which ones are hard to predict and which ones are easy?
#easy to hard NPUNID 1,20,19,16,7,31,34,34
# df_filepath = 'Input_data/Save_files/df_1_all_RFE15_RFR.xlsx'
# df_a = pd.read_excel(df_filepath, header=0)
# labels= df_a['Abundance'].copy()
# id_col= df_a['NPUNID'].copy()
# label_abund=np.ravel(labels)

# NPUNID_list=[1,20,19,16,7,31,34,43,44]
NPUNID_list=[2]
id_list=NPUNID_list
# Initialize a list to hold MSE scores for each removed ID
pearson_scores = []
r2_scores = []
mse = []

#Create filter to for dropping abundance values below
start = 0.0001
stop = 1
num_points = 15

# Calculate the common difference between consecutive terms in the series
common_diff = (math.log(stop) - math.log(start)) / (num_points - 1)

# Generate the logarithmic series as a list
Abund_filter = [start * math.exp(i * common_diff) for i in range(num_points)]

model=RandomForestRegressor(n_estimators=100)
# df_a=df_a.drop(columns=['Abundance','NPUNID'],axis=1).copy()
# df_a=RFECV_plot(df_a,label_abund,model,'a',folds=10,step=1)
# df_a=pd.concat([df_a,labels,id_col],axis=1)
print(df_a.shape)
# Loop through each ID in the list
eval_df = pd.DataFrame(columns=['Abundandce Filter', 'NPUNID', 'MSE', 'R2', 'Pearson'])
for i in Abund_filter:
    df_a= df_a[df_a['Abundance']>=i].copy()
    for id in id_list:
        df=df_a.copy()
        # Remove the row with the current ID from the dataframe
        removed_row = df.loc[df['NPUNID'] == id].copy()
        df = df.loc[df['NPUNID'] != id].copy()

        # Split the remaining data into features and target
        X_train = df.drop(['Abundance', 'NPUNID'], axis=1)
        y_train = df['Abundance']

        # Fit a random forest regression model to the training data
        model.fit(X_train, y_train)

        # Use the trained model to predict on the removed ID
        X_test = removed_row.drop(['Abundance', 'NPUNID'], axis=1)
        y_true = removed_row['Abundance']
        y_pred = model.predict(X_test)
        # Calculate the Accuracy scores and add it to the list
        pearson, _ = pearsonr(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        mse_score = mean_squared_error(y_true, y_pred)

        pearson_scores.append(pearson)
        r2_scores.append(r2)
        mse.append(mse_score)
        print(id)
        tmp=pd.DataFrame({'Abundandce Filter': i,
                                  'NPUNID':id,
                                  'MSE':mse_score,
                                  'R2':r2,
                                  'Pearson':pearson},index=[0])
        eval_df = pd.concat([eval_df,tmp],ignore_index=True,axis=0)

eval_df.to_excel('Output_data/AbundThresholdingRFR_NP(2).xlsx', index=False)
# out=pd.DataFrame(list(zip(NPUNID_list,Abund_filter,pearson_scores,r2_scores,mse)), columns=['NPUNID','Abundance Filter','Pearson', 'R2', 'MSE'])
# out.to_excel('Output_data/PredictingEachNP_RFECV.xlsx',index=False)


(765, 17)
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [9]:
##Drop BALF Samples and then Predict on them and see how well it goes
#easy to hard NPUNID 1,20,19,16,7,31,34,34
df_filepath = 'Input_data/Save_files/df_1_all_RFE50_RFR.xlsx'
df_a = pd.read_excel(df_filepath, header=0)
labels= df_a['Abundance'].copy()
id_col= df_a['NPUNID'].copy()
label_abund=np.ravel(labels)

# NPUNID_list=[1,20,19,16,7,31,34,43,44]
NPUNID_list=[7,8,9,10,11,12,13,14,15,16]
id_list=NPUNID_list
# Initialize a list to hold MSE scores for each removed ID
pearson_scores = []
r2_scores = []
mse = []

removed_row = df_a.loc[df_a['NPUNID'].isin(NPUNID_list)].copy()
df_a = df_a.loc[~df_a['NPUNID'].isin(NPUNID_list)].copy()
y_train=df_a['Abundance'].copy()
label_abund=np.ravel(y_train)
model=XGBRegressor(n_estimators=100)
df_a=df_a.drop(columns=['Abundance','NPUNID'],axis=1).copy()
df_a=RFECV_plot(df_a,label_abund,model,'a',folds=5,step=2)
print(df_a.shape)
# Loop through each ID in the list
eval_df = pd.DataFrame(columns=['Abundandce Filter', 'NPUNID', 'MSE', 'R2', 'Pearson'])
# Split the remaining data into features and target
X_train = df_a


# Fit a random forest regression model to the training data
model.fit(X_train, y_train)

# Use the trained model to predict on the removed ID
X_test = removed_row[X_train.columns]
y_true = removed_row['Abundance']
y_pred = model.predict(X_test)
# Calculate the Accuracy scores and add it to the list
pearson, _ = pearsonr(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
mse_score = mean_squared_error(y_true, y_pred)

pearson_scores.append(pearson)
r2_scores.append(r2)
mse.append(mse_score)
print(id)
tmp=pd.DataFrame({'Abundandce Filter': i,
                          'NPUNID':id,
                          'MSE':mse_score,
                          'R2':r2,
                          'Pearson':pearson},index=[0])
eval_df = pd.concat([eval_df,tmp],ignore_index=True,axis=0)

eval_df.to_excel('Output_data/PredictonBALFSamples.xlsx', index=False)
# out=pd.DataFrame(list(zip(NPUNID_list,Abund_filter,pearson_scores,r2_scores,mse)), columns=['NPUNID','Abundance Filter','Pearson', 'R2', 'MSE'])
# out.to_excel('Output_data/PredictingEachNP_RFECV.xlsx',index=False)

Recursive Feature Elimination with Correlated Features ran successfully
(21016, 28)


ValueError: Feature shape mismatch, expected: 28, got 50

In [17]:
##Run Quality Control on df to get a look at important features and such##
df_filepath = 'Input_data/Save_files/df_1_all_RFE50_RFR.xlsx'
df_a = pd.read_excel(df_filepath, header=0)
labels= df_a['Abundance'].copy()
id_col= df_a['NPUNID'].copy()
label_abund=np.ravel(labels)


# Initialize a list to hold MSE scores for each removed ID
model=RandomForestRegressor(n_estimators=100)
df_a=df_a.drop(columns=['Abundance','NPUNID'],axis=1).copy()
df_a=RFECV_plot(df_a,label_abund,model,'b',folds=5,step=2)
scram_score(df, label_abund, model, id, 0.2)
feat_drop(df, label_abund, model, id, 0.2)
# feat_drop_multifold(df, label_abund, model, id, 0.2, folds=splits)
tmp2=scorer(df, label_abund, model, id, 10)
zeros=(raw_MS_data['Abundance']==0).sum()
percent_zeros=zeros/raw_MS_data.shape[0]
tmp2['TotalZeros']=zeros
tmp2['Percent_zeros']=percent_zeros
summary_tmp.append(tmp2)
# lasso_feature_selection(df, label_abund, id)
summary=pd.concat(summary_tmp,axis=0)

Recursive Feature Elimination with Correlated Features ran successfully


NameError: name 'df' is not defined

In [19]:
id='a'
scram_score(df_a, label_abund, model, id, 0.2)
feat_drop(df_a, label_abund, model, id, 0.2)
# feat_drop_multifold(df, label_abund, model, id, 0.2, folds=splits)
tmp2=scorer(df_a, label_abund, model, id, 10)
# zeros=(raw_MS_data['Abundance']==0).sum()
# percent_zeros=zeros/raw_MS_data.shape[0]
# tmp2['TotalZeros']=zeros
# tmp2['Percent_zeros']=percent_zeros
summary_tmp.append(tmp2)
# lasso_feature_selection(df, label_abund, id)
summary=pd.concat(summary_tmp,axis=0)

C:\Users\pouls\PycharmProjects\PC_ML_using_NSP\helper_functions_KP.py:89: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(feats, rotation=90)


Scramble Scoring ran successfully


C:\Users\pouls\PycharmProjects\PC_ML_using_NSP\helper_functions_KP.py:151: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(feats, rotation=90)


Feat drop ran successfully
Scorer ran successfully


NameError: name 'summary_tmp' is not defined

In [20]:
tmp2

,pearson_mean,pearson_std,R2_mean,R2_std,MSE_mean,MSE_std,Number of Features,ID,Feature Importances,Features
0,0.954583,0.024072,0.908116,0.045036,0.273689,0.147636,26,a,"[0.04056696604843621, 0.042609126157557534, 0....","[Length, frac_aa_C, frac_aa_I, frac_aa_R, mole..."


In [ ]:
###Run Histogram of each Feature###
# Get total number of rows
df=df_a
total_count = len(df)
df = df.rename(columns=lambda x: x.replace('/', ''))
# Loop over each column of the dataframe
for col in df.columns:
    # Create a histogram of the current column, with bins=10
    data=df[col]
    plt.hist(data, weights=np.ones_like(data) / len(data))

    # Set the title of the histogram to the column name and percent of total population
    plt.title(str(col))
    plt.savefig('Output_data/{}.png'.format(str(col)))
    plt.close()
    # Show the histogram
    # plt.show()
